<a href="https://colab.research.google.com/github/oluwafemidiakhoa/AIreasearcher/blob/main/multi_agent_orchestration_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
!pip install openai aiohttp
!pip install nest_asyncio


In [48]:
import os
import asyncio
import nest_asyncio
from openai import AsyncOpenAI
from google.colab import userdata
import time
import random

nest_asyncio.apply()  # Allows nested event loops if needed


openai.api_key = userdata.get("OPENAI_API_KEY")


In [55]:
class AsyncAgent:
    def __init__(self, name, specialization, model="gpt-4-0613", max_tokens=300, temperature=0.7):
        self.name = name
        self.specialization = specialization
        self.model = model
        self.max_tokens = max_tokens
        self.temperature = temperature
        self.client = AsyncOpenAI(api_key=userdata.get("OPENAI_API_KEY"))

    async def execute(self, task_description, functions=None, max_retries=3):
        messages = [
            {"role": "system", "content": f"You are a {self.specialization} agent named {self.name}."},
            {"role": "user", "content": task_description}
        ]

        request_args = {
            "model": self.model,
            "messages": messages,
            "max_tokens": self.max_tokens,
            "temperature": self.temperature,
        }

        if functions:
            request_args["functions"] = functions
            request_args["function_call"] = "auto"

        for attempt in range(max_retries):
            try:
                response = await self.client.chat.completions.create(**request_args)

                if response.choices[0].finish_reason == "function_call":
                    return {"status": "function_call", "data": response.choices[0].message.function_call}
                else:
                    return {"status": "success", "data": response.choices[0].message.content}

            except Exception as e:
                if attempt < max_retries - 1:
                    backoff_time = 2 ** attempt + random.uniform(0, 1)
                    time.sleep(backoff_time)
                else:
                    return {"status": "error", "error": str(e)}


In [57]:
class AsyncPlannerAgent(AsyncAgent):
    def __init__(self):
        super().__init__("Planner", "planning and requirement analysis")

    async def refine_plan(self, initial_plan, feedback):
        refinement_task = f"Refine the following plan based on feedback: {feedback}\n\nPlan:\n{initial_plan}"
        refined_plan = await self.execute(refinement_task)
        return refined_plan


In [58]:
class AsyncUIDesignerAgent(AsyncAgent):
    def __init__(self):
        super().__init__("UI/UX Designer", "designing user interfaces and experiences")

    async def suggest_color_palette(self):
        task = "Suggest a color palette suitable for a handmade crafts e-commerce site. Ensure accessibility compliance."
        palette = await self.execute(task)
        return palette

    async def verify_accessibility(self, design_elements):
        task = f"Review the following design elements for accessibility compliance (WCAG 2.1): {design_elements}"
        compliance_report = await self.execute(task)
        return compliance_report


In [59]:
class AsyncBackendCoderAgent(AsyncAgent):
    def __init__(self):
        super().__init__("Backend Coder", "writing backend code and setting up databases")

    async def generate_api_docs(self, code_snippet):
        task = f"Generate API documentation for the following backend code:\n{code_snippet}"
        api_docs = await self.execute(task)
        return api_docs


In [60]:
class AsyncTesterAgent(AsyncAgent):
    def __init__(self):
        super().__init__("Tester", "creating and executing tests")

    async def suggest_test_automation(self):
        task = "Suggest an automated testing framework suitable for an e-commerce website with inventory and payment features."
        framework_suggestions = await self.execute(task)
        return framework_suggestions


In [61]:
class AsyncConductorAgent:
    def __init__(self):
        self.planner = AsyncPlannerAgent()
        self.ui_designer = AsyncUIDesignerAgent()
        self.backend_coder = AsyncBackendCoderAgent()
        self.tester = AsyncTesterAgent()

    async def orchestrate(self, user_requirements):
        # Phase 1: Planning
        print("[Conductor] Starting planning phase...")
        plan_response = await self.planner.execute(f"Create a detailed plan for: {user_requirements}")
        plan = plan_response.get("data", "Plan generation failed")
        print(f"[Conductor] Plan created:\n{plan}")

        # Phase 2: Design Suggestions
        print("[Conductor] Gathering design suggestions...")
        palette = await self.ui_designer.suggest_color_palette()
        print(f"[UI/UX Designer] Suggested color palette:\n{palette}")

        # Phase 3: Backend Code and API Docs
        print("[Conductor] Backend development...")
        backend_response = await self.backend_coder.execute(f"Develop backend code for: {user_requirements}")
        backend_code = backend_response.get("data", "Backend code generation failed")
        api_docs = await self.backend_coder.generate_api_docs(backend_code)
        print(f"[Backend Coder] API Documentation:\n{api_docs}")

        # Phase 4: Testing Strategy
        print("[Conductor] Testing phase...")
        test_plan_response = await self.tester.execute(f"Create a test plan for: {user_requirements}")
        test_plan = test_plan_response.get("data", "Test plan generation failed")
        automation_suggestions = await self.tester.suggest_test_automation()
        print(f"[Tester] Automated Testing Suggestions:\n{automation_suggestions}")

        # Aggregating results
        results = {
            "plan": plan,
            "ui_palette": palette,
            "backend_code": backend_code,
            "api_docs": api_docs,
            "test_plan": test_plan,
            "test_automation": automation_suggestions
        }

        print("[Conductor] Orchestration complete. Aggregated Results:")
        for key, value in results.items():
            print(f"{key}:\n{value}\n")

        return results


In [62]:
async def main():
    conductor = AsyncConductorAgent()
    user_requirements = (
        "I want to create an e-commerce website for selling handmade crafts, "
        "with inventory management, payment integration, and a user-friendly interface."
    )
    results = await conductor.orchestrate(user_requirements)
    print("\nFinal Results:")
    for key, value in results.items():
        print(f"{key}: {value}\n")


In [63]:
if __name__ == "__main__":
    asyncio.run(main())


[Conductor] Starting planning phase...
[Conductor] Plan created:
Creating an e-commerce website for selling handmade crafts involves multiple steps and thorough planning. Here's a comprehensive strategy:

**Step 1: Define Your Business Objectives and Target Audience**

Before creating your website, you need to clarify your business goals and identify your target audience. This will guide all other decisions in the process. 

**Step 2: Establish a Brand**

Your brand includes your business name, logo, colors, and messaging. It should reflect your business objectives, resonate with your target audience, and differentiate you from competitors.

**Step 3: Choose Your E-Commerce Platform**

There are many e-commerce platforms available, like Shopify, WooCommerce, Magento, etc. The choice depends on your budget, technical expertise, and specific needs. 

**Step 4: Website Development**

1. **Wireframing and Design:** Outline the structure of your website and create the design. This includes 